In [152]:
#IMPORTS
%run Functions.ipynb
%run ../datuslib.ipynb
import pandas as pd
import string
import numpy as np
import random
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
import time
import category_encoders as ce

from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder 

In [153]:
#CARGA DATA
#negativity = pd.read_csv("../Axel/tweets_con_negatividad.csv")
train = pd.read_csv("Data/train.csv")
#train["negativity"] = negativity["negativity"]

to_predict = pd.read_csv("Data/test.csv")

### Features y Encoder

In [154]:
train.fillna("Nothing", inplace= True)
features(train, to_predict)
keyword_in_text(train)

train.drop(columns=["id"], axis=1,inplace = True)

train.drop(columns=["location"], axis=1,inplace = True)

train['keyword'] = train['keyword'].str.replace('%',' ')
train['keyword'] = train['keyword'].str.replace('2','')
train['keyword'] = train['keyword'].str.replace('0','')
train.drop(columns=["text"], axis=1,inplace = True)

c:\users\hal9000\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [155]:
train["Tiene_key_impor"] = (train["keyword"] == "wreckage") | (train["keyword"] == "debris") | (train["keyword"] == "derailment") 
#| (train["keyword"] == "outbreak") | (train["keyword"] == "oil split")\
#| (train["keyword"] == "typhoon") | (train["keyword"] == "suicide bombing") | (train["keyword"] == "suicide bomber")

train["Tiene_key_no_impor"] = (train["keyword"] == "aftershock")

from sklearn.preprocessing import LabelEncoder

# Categorical boolean mask
categorical_feature_mask = train.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = train.columns[categorical_feature_mask].tolist()
categorical_cols

le = LabelEncoder()

# apply le on categorical feature columns
train[categorical_cols] = train[categorical_cols].apply(lambda col: le.fit_transform(col))

In [156]:
x = train[train.columns.drop("target")]
y = train['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

print(f"Original shapes: X={x.shape} y={y.shape}")
print(f"Train shapes: X={x_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={x_test.shape}  y={y_test.shape}")

Original shapes: X=(7613, 11) y=(7613,)
Train shapes: X=(6090, 11) y=(6090,)
Test  shapes: X=(1523, 11)  y=(1523,)


### XGBOOST

In [157]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=900,n_jobs=6,objective="binary:logistic",
                     max_depth= 40,min_child_weight = 1,colsample_bytree=0.5,
                     gamma=1, learning_rate=0.02, subsample=1)
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=1,
              learning_rate=0.02, max_delta_step=0, max_depth=40,
              min_child_weight=1, missing=None, n_estimators=900, n_jobs=6,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [158]:
start_time = time.time()
model.fit(x_train, y_train)

print("--- %s seconds ---" % (time.time() - start_time))

--- 6.5664873123168945 seconds ---


In [159]:
predecir(model, x_train, y_train, x_test, y_test)

Entrenamiento: 95.4351%
Testeo: 75.7058%.
F1 score: 0.7571.
   


### Predecir

### Completo Features


In [160]:
to_predict.fillna("Nothing", inplace= True)
keyword_in_text(to_predict)

ids = to_predict["id"]

to_predict.drop(columns=["id"], axis=1,inplace = True)
to_predict.drop(columns=["location"], axis=1,inplace = True)

to_predict['keyword'] = to_predict['keyword'].str.replace('%',' ')
to_predict['keyword'] = to_predict['keyword'].str.replace('2','')
to_predict['keyword'] = to_predict['keyword'].str.replace('0','')

#importance(to_predict)
to_predict.drop(columns=["text"], axis=1,inplace = True)

c:\users\hal9000\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [161]:
to_predict["Tiene_key_impor"] = (to_predict["keyword"] == "wreckage") | (to_predict["keyword"] == "debris")\
| (to_predict["keyword"] == "derailment")
#| (to_predict["keyword"] == "outbreak")\
#| (to_predict["keyword"] == "oil split") | (to_predict["keyword"] == "typhoon") \
#| (to_predict["keyword"] == "suicide bombing") | (to_predict["keyword"] == "suicide bomber")

to_predict["Tiene_key_no_impor"] = (to_predict["keyword"] == "aftershock")

# Categorical boolean mask
categorical_feature_mask = to_predict.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = to_predict.columns[categorical_feature_mask].tolist()
categorical_cols

le = LabelEncoder()

# apply le on categorical feature columns
to_predict[categorical_cols] = to_predict[categorical_cols].apply(lambda col: le.fit_transform(col))

In [162]:
best_predict = model.predict( to_predict )

### Compruebo el F1 Score

In [163]:
comprobar(best_predict)

F1 Score:  0.6969046889365614


### TUNNING

In [170]:
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

param_test = {
    'n_estimators':range(100, 1000, 100),   
    'learning_rate':(0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1)
   # 'subsample':[0.7,1],
   # 'gamma':[1,15,30],
   # 'colsample_bytree':[0.5,1.0],
  #  'max_depth':range(5,50,5)
  #  'min_child_weight' : range(1,10,1) 
}

search = GridSearchCV(
        estimator = XGBClassifier(max_depth = 40,objective="binary:logistic",colsample_bytree=0.5, gamma=1, subsample=1),
        param_grid = param_test,
        n_jobs=4,
        cv=3,
        verbose=10)


In [171]:
search.fit(x_train,y_train)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    5.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    9.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   20.7s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   36.6s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   45.4s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   57.1s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:  4.5min
[Parallel(

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.5, gamma=1,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=40, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'learning_rate': (0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
                                           0.07, 0.08, 0.09, 0.1),
                     

In [172]:
search.best_params_, search.best_score_

({'learning_rate': 0.01, 'n_estimators': 300}, 0.7279146141215107)

In [173]:
model = search.best_estimator_

In [174]:
best_predict = model.predict( to_predict )

In [175]:
comprobar(best_predict)

F1 Score:  0.7174379405455102
